In [7]:
import sys
sys.path.insert(0, '../utils')

from dataPiping import *

In [8]:
%matplotlib inline

import math
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib.dates as pltdate
import seaborn as sns
import datetime
import time
import numpy as np
import pandas as pd
from dateutil import parser
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [9]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
np.random.seed(7)

In [10]:
trainPeriod = ["2015-02-01", "2016-02-01"]

In [11]:
trainDF = pd.read_pickle('../../data/trainFebToFebCensoredMergedDF.pkl')
targetDF = pd.read_pickle('../../data/trainFebToFebTargetsMergedDF.pkl')

In [12]:
trainDF['startTime0'] = trainDF['startTime'] - makeunixtime(trainPeriod[0])

In [17]:
unixtimetostr(1448833544)

'2015-11-29 21:45:44'

In [384]:
def _discardNanStartUserTime(df,targets):
    nanStartUserTimeCust = df[df.startUserTime.isnull()].customerId.unique()
    return df[~df.customerId.isin(nanStartUserTimeCust)], targets[~targets.index.isin(nanStartUserTimeCust)]

def _calcStartUserTimeNan(g):
    isNan = g.startUserTime.isnull()
    
    if isNan.all():
        return g
    
    if isNan.any():
        timeOffset = g[~isNan].iloc[0].startUserTime - g[~isNan].iloc[0].startTime
        for i, row in g[isNan].iterrows():
            g.set_value(i, 'startUserTime', row.startTime + timeOffset)
        
    return g

def replaceNanStartUserTime(df, targets):
    df = df.groupby('customerId').apply(_calcStartUserTimeNan)
    return _discardNanStartUserTime(df, targets)

In [389]:
def normaliseData(df,targets):
    trainDF = df.copy()
    trainTargets = targets.copy()
    
    trainDF, trainTargets = replaceNanStartUserTime(trainDF, trainTargets)
    
    trainDF['startTime'] = trainDF['startTime'] - makeunixtime(trainPeriod[0])
    trainDF['startUserTime'] = (trainDF['startUserTime'] - makeunixtime(trainPeriod[0])).astype(int)
    
    uniqueCustomerId = trainDF.customerId.unique()
    customerIdEnc = preprocessing.LabelEncoder().fit(uniqueCustomerId)
    
    trainDF['customerId'] = customerIdEnc.transform(trainDF.customerId)
    trainTargets.index = customerIdEnc.transform(trainTargets.index)
    
    return trainDF, trainTargets

In [386]:
def getTrainTest():
    trainDF = pd.read_pickle('../../data/trainFebToFebCensoredMergedDF.pkl')
    targetDF = pd.read_pickle('../../data/trainFebToFebTargetsMergedDF.pkl')
    trainDF.reset_index(inplace=True)
    del trainDF['index']
    del trainDF['sessionId']
    targetSeq = targetDF.returnTime
    train, targets = normaliseData(trainDF, targetSeq)
    train_cust, test_cust, y_train, y_test = train_test_split(targets.keys(), targets, test_size=0.33, random_state=42)
    X_train = train[train.customerId.isin(train_cust)]
    X_test = train[train.customerId.isin(test_cust)]
    return X_train, X_test, y_train, y_test 

In [390]:
X_train, X_test, y_train, y_test = getTrainTest()

In [391]:
X_test

,customerId,startTime,startUserTime,sessionLength,numberdivisions,avgPrice,viewonly,changeThumbnail,imageZoom,watchVideo,view360,device,endTime,returnTime,observed
41,4,21303095,21306695.0,0.000000,1,22.000000,1,0,0,0,0,mobile,1.444052e+09,1.023290e+07,False
57,6,13108950,13094550.0,2.100000,1,24.000000,3,11,0,1,0,desktop,1.435858e+09,1.842705e+07,False
433,12,48457,48457.0,2.883333,1,17.333334,3,0,0,0,0,mobile,1.422797e+09,3.777811e+05,True
434,12,426241,426241.0,0.000000,1,30.000000,1,0,0,0,0,mobile,1.423175e+09,1.263080e+05,True
435,12,552549,552549.0,0.000000,1,5.000000,1,0,0,0,0,mobile,1.423301e+09,3.056860e+05,True
436,12,858235,858235.0,0.000000,1,60.000000,1,0,0,0,0,mobile,1.423607e+09,2.486660e+05,True
437,12,1106901,1106901.0,0.000000,1,29.000000,1,0,0,0,0,mobile,1.423856e+09,7.886500e+05,True
438,12,1895551,1895551.0,0.466667,1,28.000000,2,3,0,0,0,mobile,1.424644e+09,5.105995e+05,True
439,12,2406151,2406151.0,0.000000,1,19.500000,1,0,0,0,0,mobile,1.425155e+09,2.068153e+06,True
440,12,4474304,4474304.0,0.000000,1,33.500000,1,0,0,1,0,desktop,1.427223e+09,8.566200e+04,True


### Training sequence

use return times only

In [55]:
trainSeq = trainDF.groupby('customerId').returnTime.aggregate(lambda x: list(x))

remove censored values

In [56]:
trainSeq = trainSeq.apply(lambda x: x[0:len(x)-1])

In [57]:
targetSeq = targetDF.returnTime

Remove NaNs

In [58]:
trainSeq = trainSeq[~targetSeq.isnull()]
targetSeq = targetSeq[~targetSeq.isnull()]

In [63]:
maxLen = trainSeq.apply(lambda x: len(x)).max()

### Convert to raw arrays

In [66]:
trainSeq = trainSeq.values
targetSeq = targetSeq.values

In [68]:
trainSeq = sequence.pad_sequences(trainSeq, maxlen=maxLen)

## Train test split

In [69]:
X_train, X_test, y_train, y_test = train_test_split(trainSeq, targetSeq, test_size=0.33, random_state=42)

## Model

In [80]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=3, batch_size=64)

ValueError: The first layer in a Sequential model must get an `input_shape` or `batch_input_shape` argument.